In [ ]:
# icecuber's dummy code
# https://www.kaggle.com/icecuber/arc-1st-place-solution
dummy_run = False

from glob import glob
for fn in glob("/kaggle/input/abstraction-and-reasoning-challenge/test/*.json"):
    if "136b0064" in fn:
        print("dummy submission")
        dummy_run = True


In [ ]:
# Credit: icecuber: https://www.kaggle.com/icecuber/arc-1st-place-solution
#         Vlad & Ilia: https://www.kaggle.com/ilialar/3rd-place-end-to-end-solution

if dummy_run:
    !cp '../input/arc-subs/vlad.csv' 'base-sub.csv'
    !cp '../input/arc-subs/icecube.csv' 'filler-sub.csv'

else:
    %run ../input/arc-notebooks/3rd-place-end-to-end-solution.ipynb
    !mv submission.csv base-sub.csv
    
    %run ../input/arc-notebooks/arc_1st_place_solution.py
    !mv submission.csv filler-sub.csv    

In [ ]:
import pandas as pd
import math
import random
import numpy as np
from IPython.display import display
from random import seed

In [ ]:
seed(42)
pd.set_option('display.max_rows', 500)

In [ ]:
def standardize_sub(sub):
    sub = sub.set_index('output_id')
    sub = sub.fillna('')
    sub = sub.replace('|0|','')
    sub = sub.replace('|0| |0| |0|','')
    sub = sub.replace('|123|456|789|','')
    sub['output'] = sub['output'].apply(lambda x: x.strip())
    return sub

def count_outputs(x):
    spaces = x.count(' ')
    if (spaces > 0):
        return spaces + 1
    if len(x) > 0:
        return 1
    return 0

def dedupe(row):
    filler = [] if row['filler'] == '' else row['filler'].split(' ')
    base = [] if row['base'] == '' else row['base'].split(' ')
    for i in filler[:]:
        if i in base:
            filler.remove(i)
    return ' '.join(filler)

def combine(row):
    num_to_take = row['take']
    take = [] if row['filler_unique'] == '' else row['filler_unique'].split(' ')[:num_to_take]
    combined = take if row['base'] == '' else np.concatenate((row['base'].split(' '), take))
    return ' '.join(combined)

In [ ]:
CSV_FOLDER = './'

def fill(base_nick, filler_nick):
    base = pd.read_csv(CSV_FOLDER + base_nick + '.csv')
    base = standardize_sub(base)
    
    filler = pd.read_csv(CSV_FOLDER + filler_nick + '.csv')
    filler = standardize_sub(filler)
    
    merged = filler.merge(base,left_on='output_id', right_on='output_id',how='outer')
    merged = merged.fillna('')
    
    merged = merged.rename(columns={'output_x': 'filler', 'output_y': 'base'})
    
    merged['count_filler'] = merged['filler'].apply(count_outputs)
    merged['count_base'] = merged['base'].apply(count_outputs)
    
    merged['filler_unique'] = merged.apply(dedupe, axis=1)
    merged['count_filler_unique'] = merged['filler_unique'].apply(count_outputs)
    merged['count_dupes'] = merged['count_filler'] - merged['count_filler_unique']

    merged['need'] = merged['count_base'].apply(lambda x: 0 if x > 3 else 3 - x)
    merged['take'] = merged[['count_filler_unique','need']].min(axis=1)

    merged['combined'] = merged.apply(combine, axis=1)
    merged['count_combined'] = merged['combined'].apply(count_outputs)
    
    merged.to_csv('merged.csv')
    
    display(merged)

    sub = pd.DataFrame(merged['combined'])
    sub = sub.rename(columns={'combined': 'output'})

    sub.to_csv('submission.csv')
    
    display(sub)

In [ ]:
fill('base-sub','filler-sub')